In [1]:
using LinearAlgebra

In [2]:
N = [1 -1 -1 -1 0 0 0; 0 2 0 0 -2 -1 0;
    0 0 1 -1 1 0 0; 0 0 0 1 0 0 -1]

4×7 Matrix{Int64}:
 1  -1  -1  -1   0   0   0
 0   2   0   0  -2  -1   0
 0   0   1  -1   1   0   0
 0   0   0   1   0   0  -1

We have that $\frac{d\mathbf{X}}{dt} = \mathbf{Nv}$. Furthermore, we can rearrange this matrix into dependent and independent reactions. Such that $\mathbf{N} = [\mathbf{N_{D}}|\mathbf{N_{I}}]$.

In [3]:
order = [2, 4, 5, 1, 3, 6, 7]
N_new = N[:, order]

4×7 Matrix{Int64}:
 -1  -1   0  1  -1   0   0
  2   0  -2  0   0  -1   0
  0  -1   1  0   1   0   0
  0   1   0  0   0   0  -1

In [4]:
N_depend = N_new[:, [1, 2, 3]]
N_independ = N_new[:, [4, 5, 6, 7]]

4×4 Matrix{Int64}:
 1  -1   0   0
 0   0  -1   0
 0   1   0   0
 0   0   0  -1

We also know that the dependent fluxes would be given by
$$
\mathbf{v_{D}} = - \mathbf{N_{D}^{+}}\mathbf{N_{I}}\mathbf{v_{I}}
$$
where $\mathbf{N_{D}^{+}} = (\mathbf{N_{D}^{T}}\mathbf{N_{D}})^{-1}\mathbf{N_{D}^{T}}$

In [5]:
inv(N_depend)

LoadError: DimensionMismatch: matrix is not square: dimensions are (4, 3)

Given that $\mathbf{N_{D}}$ isn't square need pseudo inverse.

In [6]:
N_D_plus = inv(dot(transpose(N_depend), N_depend)) * N_depend
#Something wrong here

4×3 Matrix{Float64}:
  1.0   1.0  -0.0
 -2.0  -0.0   2.0
 -0.0   1.0  -1.0
 -0.0  -1.0  -0.0

In [7]:
N_D_plus2 = pinv(N_depend) #Lets take this one as the pseudo inverse

3×4 Matrix{Float64}:
 -0.52   0.24   0.48  -0.04
 -0.32  -0.16  -0.32   0.36
 -0.48  -0.24   0.52   0.04

In [8]:
# If we take an example of independent fluxes
v_I = [1 -1 0 0]
v_D = - dot(dot(N_D_plus2, N_independ), v_I)

LoadError: DimensionMismatch: first array has length 12 which does not match the length of the second, 16.

In [9]:
#Testing another matrix
N = [-1  1  0  0  0  0  0  0  0;
      1 -1 -1 -1  0  0 -1  0  0;
      0  0  0 -1  0  1  0  0  0;
      0  0  0  1 -1  0 -1  1  0;
      0  0  0  0  0  0  1 -1  0;
      0  0  0  0  1 -1  0  0  0;
      0  0  0  0  0 -1  0  0  1;
      0  0  0  0  0  1  0  0 -1]

8×9 Matrix{Int64}:
 -1   1   0   0   0   0   0   0   0
  1  -1  -1  -1   0   0  -1   0   0
  0   0   0  -1   0   1   0   0   0
  0   0   0   1  -1   0  -1   1   0
  0   0   0   0   0   0   1  -1   0
  0   0   0   0   1  -1   0   0   0
  0   0   0   0   0  -1   0   0   1
  0   0   0   0   0   1   0   0  -1

In [10]:
C = nullspace(transpose(N))

8×2 Matrix{Float64}:
 -6.05439e-17  6.33046e-34
 -6.05439e-17  6.33046e-34
  0.5          5.64356e-17
  0.5          5.74837e-18
  0.5          5.64356e-17
  0.5          9.35451e-17
  8.32667e-17  0.707107
 -8.32667e-17  0.707107

In [15]:
# More tests

N_test = [-1 1 0 0 0 1;
      1 -1 0 0 -1 0;
      1 -1 0 0 0 -1;
      -1 1 0 0 1 0;
      0 0 -1 1 0 1;
      0 0 1 -1 -1 0;
      0 0 1 -1 0 -1;
      0 0 -1 1 1 0;
      0 0 0 0 0 1;
      0 0 0 0 -1 0]
print(rank(N_test))

nullspace(transpose(N_test)) #Should I be expecting 6 conservation
#relationships?

4

10×6 Matrix{Float64}:
  0.375711    0.269114    0.0344063   -0.0849569    0.498345   -0.423875
 -0.254294    0.601201    0.0967078    0.179175     0.371379    0.210983
 -0.118306   -0.283119   -0.00633533  -0.185859     0.404485   -0.598043
 -0.748312    0.0489672   0.0559662    0.0782732    0.27752     0.0368151
 -0.233898   -0.0312507   0.60363     -0.00912692  -0.339026   -0.339005
 -0.233898   -0.0312507  -0.254373     0.489506    -0.291542   -0.452765
  0.233898    0.0312507   0.725597     0.252759     0.0409115   0.0475793
  0.233898    0.0312507  -0.132406     0.751392     0.088395   -0.0661804
 -0.0262211  -0.489732    0.0812253    0.160984     0.286078    0.212417
 -0.0262211  -0.489732    0.0812253    0.160984     0.286078    0.212417

In [16]:
nullspace(N_test) # What is this

6×2 Matrix{Float64}:
  0.0663601     0.703986
  0.0663601     0.703986
  0.703986     -0.0663601
  0.703986     -0.0663601
 -1.38778e-16   0.0
 -1.94289e-16   5.55112e-17